In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *

In [8]:
path = Path('../data')

In [35]:
#export
from lang_model import *

In [9]:
data_lm = load_data(path, 'tmp_lyrics')
data_lm.show_batch()

idx,text
0,"stay \n xxmaj knowing that you 've a weakness for the girls \n xxmaj so xxmaj mr. xxmaj bergman , xxmaj mr. b , how 'bout a cup of herbal tea \n xxmaj you 've had a stressful day , time to unwind , so unwind \n \n xxmaj no , let us talk not of films but of your life in xxmaj xxunk xxmaj hills"
1,"— the only brand my grandma smokes — and the faintest hint of xxmaj coppertone . \n \n i 'm watching xxunk circle in real close . i know you 're gon na go . \n \n xxmaj just please leave me a note . i left because you asked me to . \n \n xxmaj operator , take me home . i do n't know where"
2,shalalalala ) \n xxmaj these were our greatest hits ( xxmaj shalalalalalalala xxmaj shalalalala ) \n xxmaj the best of me and you \n \n i still remember buying you ' xxmaj band on the xxmaj run ' on the first day that we \n xxmaj kissed \n xxmaj but you always did prefer ' xxmaj mccartney i ' because it reminded you of being
3,"happen to you \n xxmaj wake up and stumble , trip and fall \n xxmaj you had so much \n xxmaj and now you 've lost it all \n \n xxmaj busted down and broken all in two \n xxmaj but you never thought \n this would happen to you \n xxmaj wake up and stumble , trip and fall \n xxmaj"
4,"stole the show \n xxmaj long ago , long ago \n xxmaj nights turned slow \n xxmaj thunder would roll \n \n xxmaj winters rise white as snow \n xxmaj right as i turn around to go \n xxmaj the wind blows , the wind blows \n xxmaj caught in the cold \n a wheel for my shoulder \n \n xxmaj"


In [12]:
trn_dl = data_lm.train_dl
val_dl = data_lm.valid_dl

In [11]:
#export
def lm_loss(input, target, kld_weight=0):
    sl, bs = target.size()
    sl_in,bs_in,nc = input.size()
    return F.cross_entropy(input.view(-1,nc), target.view(-1))

In [12]:
#export
def bn_drop_lin(n_in, n_out, bn=True, initrange=0.01,p=0, bias=True, actn=nn.LeakyReLU(inplace=True)):
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    linear = nn.Linear(n_in, n_out, bias=bias)
    if initrange:linear.weight.data.uniform_(-initrange, initrange)
    if bias: linear.bias.data.zero_()
    layers.append(linear)
    if actn is not None: layers.append(actn)
    return layers

In [10]:
learn = language_model_learner(data_lm, arch=TransformerXL)
#learn.load('lyrics_fine_tuned_novel')

/home/ubuntu/projects/fastai/lib/python3.6/site-packages/fastai/text/learner.py:209: UserWarning: There are no pretrained weights for that architecture yet!
  warn("There are no pretrained weights for that architecture yet!")


In [11]:
encoder = deepcopy(learn.model[0])
encoder

TransformerXL(
  (encoder): Embedding(60004, 410)
  (pos_enc): PositionalEncoding()
  (drop_emb): Dropout(p=0.1)
  (layers): ModuleList(
    (0): DecoderLayer(
      (mhra): MultiHeadRelativeAttention(
        (attention): Linear(in_features=410, out_features=1230, bias=False)
        (out): Linear(in_features=410, out_features=410, bias=False)
        (drop_att): Dropout(p=0.1)
        (drop_res): Dropout(p=0.1)
        (ln): LayerNorm(torch.Size([410]), eps=1e-05, elementwise_affine=True)
        (r_attn): Linear(in_features=410, out_features=410, bias=False)
      )
      (ff): SequentialEx(
        (layers): ModuleList(
          (0): Linear(in_features=410, out_features=2100, bias=True)
          (1): ReLU(inplace)
          (2): Dropout(p=0.1)
          (3): Linear(in_features=2100, out_features=410, bias=True)
          (4): Dropout(p=0.1)
          (5): MergeLayer()
          (6): LayerNorm(torch.Size([410]), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (1): 

In [13]:
x, y = next(iter(trn_dl))
x.size(), y.size()

(torch.Size([64, 70]), torch.Size([64, 70]))

In [14]:
outs = encoder(x)

In [28]:
outs[-1][-1].size()

torch.Size([64, 70, 410])

In [26]:
[out.size() for out in outs[-1]]

[torch.Size([64, 70, 410])]

In [17]:
generator = deepcopy(learn.model) 

In [18]:
generator.load_state_dict(learn.model.state_dict())

In [19]:
#export
class TextDicriminator(nn.Module):
    def __init__(self,encoder, nh, bn_final=True):
        super().__init__()
        #encoder
        self.encoder = encoder
        #classifier
        layers = []
        layers+=bn_drop_lin(nh*3,nh,bias=False)
        layers += bn_drop_lin(nh,nh,p=0.25)
        layers+=bn_drop_lin(nh,1,p=0.15,actn=nn.Sigmoid())
        if bn_final: layers += [nn.BatchNorm1d(1)]
        self.layers = nn.Sequential(*layers)
    
    def pool(self, x, bs, is_max):
        f = F.adaptive_max_pool1d if is_max else F.adaptive_avg_pool1d
        return f(x.permute(0,2,1), (1,)).view(bs,-1)
    
    def forward(self, inp,y=None):
        raw_outputs, outputs = self.encoder(inp)
        output = outputs[-1]
        bs,sl,_ = output.size()
        avgpool = self.pool(output, bs, False)
        mxpool = self.pool(output, bs, True)
        x = torch.cat([output[:,-1], mxpool, avgpool], 1)
        out = self.layers(x)
        return out

In [20]:
disc = TextDicriminator(encoder,400).cuda()

In [21]:
optimizerD = optim.Adam(disc.parameters(), lr = 3e-4)
optimizerG = optim.Adam(generator.parameters(), lr = 3e-3, betas=(0.7, 0.8))

In [22]:
#export
def seq_gumbel_softmax(input):
    samples = []
    bs,sl,nc = input.size()
    for i in range(sl): 
        z = F.gumbel_softmax(input[:,i,:])
        samples.append(torch.multinomial(z,1))
    samples = torch.stack(samples).transpose(1,0).squeeze(2) 
    return samples

In [23]:
#export
from tqdm import tqdm

In [29]:
#export
def reinforce_loss(input,sample,reward):
    loss=0
    bs,sl = sample.size()
    for i in range(sl):
        loss += -input[:,i,sample[:,i]] * reward
    return loss/sl

In [4]:
#export
def step_gen(ds,gen,disc,optG,crit=None):
    gen.train(); disc.train()
    x,y = ds
    bs,sl = x.size()
    fake,_,_ = gen(x)
    gen.zero_grad()
    fake_sample =seq_gumbel_softmax(fake)
    with torch.no_grad():
        gen_loss = reward = disc(fake_sample)
        if crit: gen_loss = crit(fake,fake_sample,reward.squeeze(1))
        gen_loss = gen_loss.mean()
    gen_loss.requires_grad_(True)
    gen_loss.backward()
    optG.step()
    return gen_loss.data.item()

In [5]:
#export
def step_disc(ds,gen,disc,optD,d_iters):
    for j in range(d_iters):
        gen.eval(); disc.train()
        with torch.no_grad():
            fake,_,_ = gen(x)
            fake_sample = seq_gumbel_softmax(fake)
        disc.zero_grad()
        fake_loss = disc(fake_sample)
        real_loss = disc(y.view(bs,sl))
        disc_loss = (fake_loss-real_loss).mean(0)
        disc_loss.backward()
        optimizerD.step()
    return disc_loss.data.item()

In [1]:
#export
def evaluate(ds,gen,disc,crit=None):
    with torch.no_grad():
        x, y = ds
        bs,sl = x.size()
        fake,_,_ = gen(x)
        fake_sample =seq_gumbel_softmax(fake)
        gen_loss = reward = disc(fake_sample)
        if crit: gen_loss = crit(fake,fake_sample,reward.squeeze(1))
        gen_loss = gen_loss.mean()
        fake_sample = seq_gumbel_softmax(fake)
        fake_loss = disc(fake_sample).mean(0).view(1).data.item()
        real_loss = disc(y.view(bs,sl)).mean(0).view(1).data.item()
        disc_loss = (fake_loss-real_loss).mean(0).view(1).data.item()
    return fake,gen_loss,disc_loss,fake_loss

In [24]:
#export
def train(gen, disc, epochs, trn_dl, val_dl, optimizerD, optimizerG, crit=None,first=True):
    gen_iterations = 0
    
    for epoch in range(epochs):
        
        gen.train(); disc.train()
        n = len(trn_dl)
        #train loop
        with tqdm(total=n) as pbar:
            for i, ds in enumerate(trn_dl):
                gen_loss = step_gen(ds,gen,disc,optimizerG,crit)
                gen_iterations += 1
                d_iters = 3
                disc_loss = step_disc(ds,gen,disc,optimizerD,d_iters)
                pbar.update()
        print(f'Epoch {epoch}:')
        print('Train Loss:')
        print(f'Loss_D {disc_loss}; Loss_G {gen_loss} Ppx {torch.exp(lm_loss(fake,y))}')
        print(f'D_real {real_loss}; Loss_D_fake {fake_loss}')
        
        disc.eval(), gen.eval()
        with tqdm(total=len(val_dl)) as pbar:
            for i, ds in enumerate(val_dl):
                fake,gen_loss,disc_loss,fake_loss = evaluate(ds,gen,disc,crit)
                pbar.update()
        print('Valid Loss:')
        print(f'Loss_D {disc_loss}; Loss_G {gen_loss} Ppx {torch.exp(lm_loss(fake,ds[-1]))}')
        print(f'D_real {real_loss}; Loss_D_fake {fake_loss}')

In [39]:
#export
nh = {'AWD':400,'XL':410}
crits={'gumbel':None,'reinforce':reinforce_loss}

#train a language model with gan objective
def run(path,filename,pretrained,model,crit=None,preds=True,epochs=6):
    
    #load data after running preprocess
    print(f'loading data from {path}/{filename};')
    data_lm = load_data(path, filename)
    trn_dl = data_lm.train_dl
    val_dl = data_lm.valid_dl
    
    #select encoder for model
    print(f'training text gan model {model}; pretrained from {pretrained};')
    learn = language_model_learner(data_lm, arch=models[model])
    learn.load(pretrained)
    encoder = deepcopy(learn.model[0])
    
    generator = deepcopy(learn.model)
    generator.load_state_dict(learn.model.state_dict())
    disc = TextDicriminator(encoder,nh[model]).cuda()
    
    disc.train()
    generator.train()
    
    #create optimizers
    optimizerD = optim.Adam(disc.parameters(), lr = 3e-4)
    optimizerG = optim.Adam(generator.parameters(), lr = 3e-3, betas=(0.7, 0.8))
    
    print(f'training for {epochs} epochs')
    train(generator, disc, epochs, trn_dl, val_dl, optimizerD, optimizerG, first=False)
    
    #save model
    learn.model.load_state_dict(generator.state_dict())
    print(f'saving model to {path}/{filename}_{model}_gan_{crit}')
    learn.save(filename+'_'+model+'_gan_'+crit)
    
    #generate output from validation set
    if preds:
        print(f'generating predictions and saving to {path}/{filename}_{model}_preds.txt;')
        get_valid_preds(learn,data_lm,filename+'_'+model+'_preds.txt')

In [ ]:
#export
if __name__ == '__main__': fire.Fire(run)

In [1]:
!/home/ubuntu/projects/creativity-model-zoo/notebooks/notebook2script.py textgan.ipynb

Converted textgan.ipynb to ../textgan.py
